# Questions to Answer

## - The stakeholder's first question is: does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

- They want you to perform a statistical test to get a mathematically-supported answer.
- They want you to report if you found a significant difference between ratings.
- If so, what was the p-value of your analysis?
- And which rating earns the most revenue?
- They want you to prepare a visualization that supports your finding.

## - It is then up to you to think of 2 additional hypotheses to test that your stakeholder may want to know.

    Some example hypotheses you could test:

- Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?
- Do movies released in 2020 earn less revenue than movies released in 2018?
- How do the years compare for movie ratings?
- Do some movie genres earn more revenue than others?
- Are some genres higher rated than others? etc.

# imports functions and connection

In [10]:
# imports
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm_notebook
import glob
import json, os, time, math
import tmdbsimple as tmdb

In [11]:
with open('/Users/marcolaureano2/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [12]:
tmdb.API_KEY =  login['api-key']

In [13]:
def get_movie_with_rating(movie_id):
    # get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            # save a 'certification' key in info with the certification
            info['certification'] = c['certification']
    return info

In [14]:
def write_json(new_data, filename): 

    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [15]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['IMDB Movie Dataset Info.docx',
 'api_resultsfinal_tmdb_data_2001.csv.gz',
 'tmdb_results_combined_2002.csv.gz',
 '.DS_Store',
 'tmdb_results_combined_2012.csv.gz',
 'tmdb_results_combined_2016.csv.gz',
 'tmdb_api_results_2016.json',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2002.csv.gz',
 'tmdb_results_combined_2014.csv.gz',
 'api_resultstmdb_api_results_[2001, 2002].json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_[2001, 2002].json',
 'tmdb_api_results_2014.json',
 'tmdb_api_results_2015.json',
 'TitleRatings.tsv',
 'tmdb_api_results_2003.json',
 'title-akas-us-only.csv',
 'tmdb_results_combined_2013.csv.gz',
 'filtered_ratings.csv',
 'tmdb_results_combined_2001.csv.gz',
 'preprocessed_movie_data.csv',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'tmdb_api_results_2012.json',
 'TitleBasics.tsv',
 'Data',
 'tmdb_results_combined_2015.csv.gz',
 'tmdb_api_results_2013.json']

In [16]:
basics = pd.read_csv('/Users/marcolaureano2/Documents/GitHub/Movie-Database/Data/preprocessed_movie_data.csv')
basics

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,81515,tt0083324,movie,The White Lions,The White Lions,0,NaN,NaN,97,Drama
4,86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...
84543,10016225,tt9915130,movie,Meet John Doe,Meet John Doe,0,NaN,NaN,120,NaN
84544,10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
84545,10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
84546,10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


# 2012, 2013, 2014

In [8]:
YEARS_TO_GET = [2012, 2013, 2014]
errors = [ ]

In [9]:
# OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for {YEAR}.")
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    df = basics.loc[ basics['startYear'] == YEAR].copy()
    movie_ids = df['tconst'].copy()
    movie_ids
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}", position = 1, leave=True):
        try:
    # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
    # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
    # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        
        except Exception as e:
            errors.append([YEARS_TO_GET, e])
            continue
        
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}tmdb_results_combined_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/3 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/1246 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/2973 [00:00<?, ?it/s]

Movies from 2014:   0%|          | 0/4968 [00:00<?, ?it/s]

# 2015, 2016

In [17]:
YEARS_TO_GET = [2015, 2016]

In [18]:
# OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        print(f"Creating {JSON_FILE} for API results for {YEAR}.")
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    df = basics.loc[ basics['startYear'] == YEAR].copy()
    movie_ids = df['tconst'].copy()
    movie_ids
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}", position = 1, leave=True):
        try:
    # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
    # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
    # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        
        except Exception as e:
            errors.append([YEARS_TO_GET, e])
            continue
        
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}tmdb_results_combined_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2015:   0%|          | 0/5116 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5326 [00:00<?, ?it/s]

# glob

In [ ]:
print(f"- Total errors: {len(errors)}")

In [ ]:
final_year_df

In [ ]:
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("Data/final_tmdb_data*.csv.gz"))
tmdb_files